In [1]:
from dataset import CANDataset
import torch
from torchvision import transforms, datasets

In [2]:
train_dataset = CANDataset(
    root_dir='../Data/TFRecord_w29_s15/1/',
    window_size=29, is_train=True)
# train_dataset.total_size = 1000
train_sampler = None
train_loader = torch.utils.data.DataLoader(
    train_dataset, batch_size=256, shuffle=False,
    num_workers=2, pin_memory=True, sampler=train_sampler)

In [12]:
val_dataset = CANDataset(
    root_dir='../Data/TFRecord_w29_s15/1/',
    window_size=29, is_train=False)
val_loader = torch.utils.data.DataLoader(
    val_dataset, batch_size=2048, shuffle=False,
    num_workers=8, pin_memory=True, sampler=None)

# Baseline Model

In [15]:
from networks.inception import InceptionResnet
from networks.simple_cnn import BaselineCNNClassifier

In [16]:
# model = InceptionResnet(n_classes = 5)
baseline_model = BaselineCNNClassifier(n_classes=5)

In [18]:
model_path = './save/cnn_lr0.001_bs1024_032522_100842_cosine_warm/models/last.pth'
ckpt = torch.load(model_path)
state_dict = ckpt['model']
baseline_model.load_state_dict(state_dict=ckpt['model'])

<All keys matched successfully>

In [19]:
import numpy as np
total_pred = np.array([], dtype=int)
total_label = np.array([], dtype=int)

baseline_model = baseline_model.cuda()
baseline_model.eval()
with torch.no_grad():
    for images, labels in tqdm.tqdm(val_loader):
        images = images.cuda(non_blocking=True)
        outputs = baseline_model(images)
        _, pred = outputs.topk(1, 1, True, True)
        pred = pred.t().cpu().numpy().squeeze(0)
        total_pred = np.concatenate((total_pred, pred), axis=0)
        total_label = np.concatenate((total_label, labels), axis=0)

 20%|██        | 33/162 [00:13<00:47,  2.74it/s]Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7f64bbbdb9e0>
Traceback (most recent call last):
  File "/home/dhkim1/miniconda3/envs/torch/lib/python3.7/site-packages/torch/utils/data/dataloader.py", line 1328, in __del__
    self._shutdown_workers()
  File "/home/dhkim1/miniconda3/envs/torch/lib/python3.7/site-packages/torch/utils/data/dataloader.py", line 1320, in _shutdown_workers
    if w.is_alive():
  File "/home/dhkim1/miniconda3/envs/torch/lib/python3.7/multiprocessing/process.py", line 151, in is_alive
    assert self._parent_pid == os.getpid(), 'can only test a child process'
AssertionError: can only test a child process
Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7f64bbbdb9e0>
Traceback (most recent call last):
  File "/home/dhkim1/miniconda3/envs/torch/lib/python3.7/site-packages/torch/utils/data/dataloader.py", line 1328, in __del__
    self._shutdown_workers()
  Fil

In [20]:
from utils import cal_metric
cm, results = cal_metric(total_label, total_pred)
results

{'fnr': array([0.00635346, 0.08747295, 0.16665375, 0.07131349, 0.07489008]),
 'rec': array([0.99993647, 0.99912527, 0.99833346, 0.99928687, 0.9992511 ]),
 'pre': array([0.99947242, 0.99990785, 0.99957315, 0.99981502, 0.99987913]),
 'f1': array([0.99970439, 0.99951641, 0.99895292, 0.99955087, 0.99956502])}

# SupCon model

In [4]:
import torch.optim as optim

In [5]:
from networks.simple_cnn import SupConCNN, LinearClassifier

model = SupConCNN(feat_dim=128)
classifier = LinearClassifier(n_classes=5)

In [7]:
model_path = './save/SupCon_cnn_lr0.001_bs1024_032522_174259_cosine_warm/models/last.pth'
ckpt = torch.load(model_path)
state_dict = ckpt['model']
model.load_state_dict(state_dict=ckpt['model'])

<All keys matched successfully>

In [9]:
class_model_path = './save/SupCon_cnn_lr0.001_bs1024_032522_174259_cosine_warm/models/last_classifier.pth'
ckpt = torch.load(class_model_path)
state_dict = ckpt['model']
classifier.load_state_dict(state_dict=state_dict)

<All keys matched successfully>

In [13]:
import numpy as np
import tqdm
total_pred = np.array([], dtype=int)
total_label = np.array([], dtype=int)

model = model.cuda()
classifier = classifier.cuda()
model.eval()
classifier.eval()
with torch.no_grad():
    for images, labels in tqdm.tqdm(val_loader):
        images = images.cuda(non_blocking=True)
        outputs = classifier(model.encoder(images))
        _, pred = outputs.topk(1, 1, True, True)
        pred = pred.t().cpu().numpy().squeeze(0)
        total_pred = np.concatenate((total_pred, pred), axis=0)
        total_label = np.concatenate((total_label, labels), axis=0)

100%|██████████| 162/162 [00:54<00:00,  2.99it/s]


In [14]:
from utils import cal_metric
cm, results = cal_metric(total_label, total_pred)
results

{'fnr': array([0.00146618, 0.06445375, 0.09689171, 0.06338977, 0.06764265]),
 'rec': array([0.99998534, 0.99935546, 0.99903108, 0.9993661 , 0.99932357]),
 'pre': array([0.9995701 , 1.        , 0.99996121, 0.99992072, 0.99995165]),
 'f1': array([0.99977768, 0.99967763, 0.99949593, 0.99964333, 0.99963751])}

In [7]:
def extract_embedding(model, data_loader):
    model = model.cuda()
    embs = torch.empty(0, 576, dtype=torch.double).cuda()
    for data, labels in data_loader:
        data = data.cuda()
        with torch.no_grad():
            feats = model.encoder(data)
            embs = torch.cat((embs, feats), dim=0)
    return embs

In [12]:
model = model.cuda()
classifier = LinearClassifier(n_classes=5)
classifier = classifier.cuda()
criterion = torch.nn.CrossEntropyLoss()
optimizer = optim.SGD(classifier.parameters(), lr=0.01)

In [10]:
from SupContrast.util import AverageMeter
from SupContrast.util import accuracy
from tqdm import tqdm

In [13]:
epochs = 10
for epoch in range(epochs):
    model.eval()
    classifier.train()
    
    losses = AverageMeter()
    accs = AverageMeter()
    
    for data, labels in tqdm(train_loader):
        data = data.cuda()
        labels = labels.cuda()
        bsz = labels.shape[0]
        with torch.no_grad():
            feats = model.encoder(data)
            
        output = classifier(feats.detach())
        loss = criterion(output, labels)
        acc = accuracy(output, labels, topk=(1, ))
        losses.update(loss.item(), bsz)
        accs.update(acc[0], bsz)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    print(f'Epoch {epoch}: loss = {losses.avg}, acc = {accs.avg}')

100%|██████████| 3021/3021 [08:14<00:00,  6.12it/s]


Epoch 0: loss = 0.013830464898798547, acc = tensor([99.7984], device='cuda:0')


100%|██████████| 3021/3021 [08:11<00:00,  6.14it/s]


Epoch 1: loss = 0.004980118928648514, acc = tensor([99.8779], device='cuda:0')


100%|██████████| 3021/3021 [08:12<00:00,  6.13it/s]


...

Epoch 7: loss = 0.0025286218821368844, acc = tensor([99.9384], device='cuda:0')


100%|██████████| 3021/3021 [08:11<00:00,  6.15it/s]


Epoch 8: loss = 0.002394688842020142, acc = tensor([99.9426], device='cuda:0')


100%|██████████| 3021/3021 [08:13<00:00,  6.12it/s]

Epoch 9: loss = 0.0022813792193305577, acc = tensor([99.9459], device='cuda:0')
